# SparK Fortress Inc. Data Analysis

### PART ONE: Data Ingestion, Integration and Quality Issues <br>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import csv
import datetime
import math
import numpy as np

#### Question 1

In [ ]:
df = pd.read_csv('SparkFortress.csv', skiprows = lambda x: x in [0,1], parse_dates = True, decimal = '.',thousands = ',')
tdate = datetime.datetime.now()
tdate = tdate.isoformat()
for index ,row in df.iterrows():
    pp=''
    df.loc[index,'Salary']=int(pp.join(e for e in str(df.loc[index,'Salary']).split()[0][:-2] if e.isalnum()))
    df.loc[index,'time_tenured'] = pd.to_datetime(tdate) - pd.to_datetime(df.loc[index,'Start_date'])
    df.loc[index,'time_tenured'] =int(int(str(df.loc[index,'time_tenured']).split()[0])/365.25)
    if df.loc[index,'Ethnicity'] != 'White':
        df.loc[index,'Ethnicity_Modified'] = 'Other'
    else:
        df.loc[index,'Ethnicity_Modified'] = 'White'
    
newDataFrame = df.drop(['Start_date','Surname','Name'], axis = 1)
newDataFrame.columns = ['Age','Gender', 'Nationality','Ethnicity','Dept.','Position','Salary','Years_Tenured','Ethnicity_Modified']

mask = newDataFrame['Dept.'] == 'Software Engineering     '
newDataFrame.loc[mask,'Dept.'] = 'Software Engineering'

newDataFrame.head(5)

#### Question 2: 
#### a.) <p>The first setup is called data warehousing and it is basically putting the data together into one repository. data wareshousing is easy way for data integration as only one-way-mapping is needed. but you have to have control over the queries and the limits of this method is predictable. the second setup is in-place intergration which is keeping the data where it is and sending queries to get the data. it is more complex as it needs two-way mapping but it is better than data warehousing when it comes to dealing with dynamic data. In our case, the data is not dynamic and relatively simple so data warehousing is a better choice</p>
#### b.) <p>there are some challenges in data integration like data extraction and cleaning (which is getting data in a structured form and how to deal with web sources to extract the data from), accessing sources limitations(eg. number of queries limits, sources access limits and limits on the type of the query), and schema mapping(which schema is better for our case, matching between the local schema and the global schemas). </p>
#### c.) <p>in our dataset, the data quality issues that can arise might be on an attribute level (eg. missing values, misfielded values, appriviations) or on a record level (eg. dupliacated records, contracting records, wrong attribute dependencies). also might be a source problem like wrong referencing in the data</p>

### PART TWO:  Exploratory Data Analysis
#### Question 3

In [ ]:
NatBox = px.box(newDataFrame,x="Nationality", y= "Salary",points='all',
                hover_data= ['Position'], title='Average Salary based on the Nationality')

GenderBox = px.box(newDataFrame,x="Gender", y= "Salary",points='all',
                   hover_data= ['Age' , 'Position'],title='Average Salary based on the Gender')

EthnicityBox = px.box(newDataFrame,x="Ethnicity", y= "Salary",points='all' ,hover_data = ['Gender']
                      ,title='Average Salary based on the Ethnicity')

DeptBox = px.box(newDataFrame,x="Dept.", y= "Salary" ,points='all',hover_data = ['Position','Years_Tenured']
                 ,title='Average Salary based on the Department')


In [ ]:
NatBox.show()

<p>As we can see in the box and whisker plot above that the US employess are the highest in every thing; in numbers in average salary and in max salary. but we can't say that there is a discremenation based on that plot because this plot addresses only one aspect of the data. Also, as we see that most of the other nationalities contains only one employee so there is no way of deciding whither there is any kind of discremenation in the company.</p>

In [ ]:
GenderBox.show()

<p>As we can see from the gender plot is the distribution of the point is some how similar but the highest paid employee is a female. But looking at this point we can see that she is the CEO of the company so it is reasonable that her salary is higher than everyone else. Ignoring the CEO point we can see that the limits and the distribution of the points on box genders is close to each other. And again we can not judge if there is discrimination based on one aspect, we have to see all the aspects to get the final discision.</p>

In [ ]:
EthnicityBox.show()

<p>As shown in the figure that the white race is the highest in the company then comes the african american or black race. But addressing the salary based only on the race does not conduct any result about wither there is discrimination or not. moreover, looking at the figure we see most of the non-white races are in the salary range of 100k-120k; which is considered a good salary in the company. but we still can not judge based on this figure alone.</p>

In [ ]:
DeptBox.show()

<p>Still, the CEO is the highest. But, here the distribution is interesting and reveales some information about the company, like the production department has the highest amount of employess and the salary ranges is very large as we can tell from the number of outliers in the production department. the highest position in every department is the directore of the department. but still, we can not judge on discrimination in the company by only looking to the departments distribution and also it is known that there is positions that are more critical than other positions so the salaries will be different.</p>

#### Question 4
#### a.)

In [ ]:
ageHist = px.histogram(newDataFrame, x ='Age',histfunc='count',range_x =[25,70],
                       nbins=9, title='Count of the employees based on the Age ranges')

tenureHist = px.histogram(newDataFrame, x ='Years_Tenured',histfunc='count',
                          range_x =[int(newDataFrame['Years_Tenured'].min()),int(newDataFrame['Years_Tenured'].max()+1)],
                          nbins=7, color='Dept.', title='Count of employees based on the tenure years ranges')

In [ ]:
ageHist.show()

<p>looking at the histogram above we see that the highest number of employees is in range of 30-35 years which is 48 employees. then the numbers starting to get slowly lower as the age range gets older until we reach the age range 65-70 which contains only one employee. Also, we can see that the young employees are hard to get accepted in the company as there is only 28 employees in the age range of 25-30. And it just a coincidence that these young employees are proposing for a position that has many employees already. to conclode, the age distribution is normal and rational.</p>

In [ ]:
tenureHist.show()

<p>As shown in the figure above that the  IT dept.  expanded in the past two years and it contains most of the freshest employees in the company. And the admin offices got a major change recenlty. As well as in the production Dept that they add a large amount of employees every two years and the department expands rapidly over the years as shown in the figure. Looking at the oldest employee in the company with tenure time of 14 years we can see that he/she is from the sales Dept.  </p>

#### b.)

In [ ]:
ageBoxBins=[]
tenuredBoxBins = []
for i in range(newDataFrame['Age'].min(),newDataFrame['Age'].max()+3 , 5):
    ageBinRange=newDataFrame[newDataFrame['Age'].isin(np.arange(i, i+5))]
    ageBoxBins.append(ageBinRange)
    
for i in range (newDataFrame['Years_Tenured'].min(), newDataFrame['Years_Tenured'].max(), 2):
    tenuredBinRange = newDataFrame[newDataFrame['Years_Tenured'].isin(np.arange(i, i+2))]
    tenuredBoxBins.append(tenuredBinRange)

ageSalaryBox = go.Figure()
tenuredSalaryBox = go.Figure()

for i in range(len(ageBoxBins)):
    ageSalaryBox.add_trace(go.Box(y=ageBoxBins[i]['Salary'],name=('Ages from '+str(25+i*5)+':'+str(29+i*5)),
                                  boxpoints= 'all',boxmean =True ))
    
for i in range (len(tenuredBoxBins)):
    tenuredSalaryBox.add_trace(go.Box(y=tenuredBoxBins[i]['Salary'],name=('Years Tenured from '+str(4+i*2)+':'+str(6+i*2)),
                                      boxpoints= 'all',boxmean =True))


In [ ]:
ageSalaryBox.update_layout(title='Average Salary per Age Ranges ', xaxis_title='Age Ranges', yaxis_title='Average Salary')

ageSalaryBox.show()

<p>Looking at the figure above we see that the age range 60-64 got the highest average salary but we cant say there is a discrimination becuase this age range only contains two employees. on the other hand, we see the oldest age range got a really low salary but as the same reason we can'y judge because this range contains only one employee and he/she might have a small job in the company. Another observation from the figure that most of the age ranges' average Salary are split into 2, a low one and a high one; that shows that in every age range there is employees with small jobs and employees with critical jobs. One othe thing to get from the plot, is that the mean Salary from the age ranges from 25-54 are in the Salary range of 60k-80k.</p>

In [ ]:
tenuredSalaryBox.update_layout(title='Average Salary per Tenure years Ranges ', xaxis_title='Tenure years Ranges', yaxis_title='Average Salary')

tenuredSalaryBox.show()

<p>Looking at the plot above we see an interesting thing which that the oldest employees in the company are among the lowest paid employees. Also, like mentioned in the Age plot that the employees tenure years ranges' Salary is split into 2 a high one and a low one and the lower part contains more employees than the higher part except the one with range of 4-6 years it is spread evenly with a slightly higher number at the maximum value. And, the mean salary of all the ranges until the range 10-12 years is in between the salary range of 60k-90k.</p>

#### c.)

In [ ]:
pos1=newDataFrame[newDataFrame['Position'].eq('Production Technician I')].describe()
print ('Position: Production Technician I')
print(pos1[['Salary']])
pos2 = newDataFrame[newDataFrame['Position'].eq('Software Engineer')].describe()
print ('\nPosition: Software Engineer ')
print(pos2[['Salary']])
pos3 = newDataFrame[newDataFrame['Position'].eq('Network Engineer')].describe()
print ('\nPosition: Network Engineer')
print(pos3[['Salary']])

### PART THREE: Hypothesis Testing
#### a.

In [ ]:
maleDf = newDataFrame[newDataFrame['Gender'].eq('Male')].sort_values(by=['Age'])
maleDf = maleDf.reset_index(drop=True)
femaleDf = newDataFrame[newDataFrame['Gender'].eq('Female')].sort_values(by=['Age'])
femaleDf = femaleDf.reset_index(drop=True)

maleS=(len(maleDf)-1)*maleDf['Salary'].var()
femaleS = (len(femaleDf)-1)*femaleDf['Salary'].var()
DegOfFree = len(maleDf)+len(femaleDf)-2

PooledVar = (maleS+femaleS)/DegOfFree
tDenominator = math.sqrt(PooledVar/len(maleDf) + PooledVar/len(femaleDf))
T_Score = ((maleDf['Salary'].mean() - femaleDf['Salary'].mean())-0)/tDenominator

<p>This project was mainly testing if there is some kind of discrimination in the company so the null hypothesis was: µ MaleSalary - µ FemaleSalary = 0. so the goal is to test wither this hupothesis is true or that µ MaleSalary - µ FemaleSalary != 0. So after splitting the data into 2 sup datasets for both genders which are independant and we do not know the variance of each dataset we started the T-test. First by getting the pooled variance to the dataset. Then we started the T-testing and the T-score was equal to = 1.260662. With Degrees of freedom = 172 and the hypothesis is two-tailed we applied these values to the online p-value calculator and the p-value was equal : 0.20916 which is not significant at 'p less than 0.05' </p>

#### b.

In [ ]:
maleDfYoung = maleDf[maleDf['Age'].between(0,34)]
maleDfYoung = maleDfYoung.reset_index(drop=True)
maleDfOld = maleDf[maleDf['Age'].between(35,70)]
maleDfOld = maleDfOld.reset_index(drop=True)
femaleDfYoung = femaleDf[femaleDf['Age'].between(0,34)]
femaleDfYoung = femaleDfYoung.reset_index(drop=True)
femaleDfOld = femaleDf[femaleDf['Age'].between(35,70)]
femaleDfOld = femaleDfOld.reset_index(drop=True)

maleYoungS=(len(maleDfYoung)-1)*maleDfYoung['Salary'].var()
femaleYoungS = (len(femaleDfYoung)-1)*femaleDfYoung['Salary'].var()
DegOfFreeYoung = (len(maleDfYoung)+len(femaleDfYoung))-2

PooledVarYoung = (maleYoungS+femaleYoungS)/DegOfFreeYoung
tDenominatorYoung = math.sqrt(PooledVarYoung/len(maleDfYoung) + PooledVarYoung/len(femaleDfYoung))
T_ScoreYoung = ((maleDfYoung['Salary'].mean() - femaleDfYoung['Salary'].mean())-0)/tDenominatorYoung
#------------------------------------------------------
maleOldS=(len(maleDfOld)-1)*maleDfOld['Salary'].var()
femaleOldS = (len(femaleDfOld)-1)*femaleDfOld['Salary'].var()
DegOfFreeOld = (len(maleDfOld)+len(femaleDfOld))-2

PooledVarOld = (maleOldS+femaleOldS)/DegOfFreeOld
tDenominatorOld = math.sqrt(PooledVarOld/len(maleDfOld) + PooledVarOld/len(femaleDfOld))
T_ScoreOld = ((maleDfOld['Salary'].mean() - femaleDfOld['Salary'].mean())-0)/tDenominatorOld

print(T_ScoreYoung)
print(T_ScoreOld)

<p>Here we started to make more detailed testing by adding the Age factor. We splitted the supdatasets to 4 more subdatasets so each supdataset contains 2 more subdatasets one for employees less than 35 years with degrees of freedom euqal "74" and one for employees more than 35 years with degrees of freedom equal "96". Then we applied 2 T-tests, one for the Gender less than 35 years and other for Gender more than 35 years. and the resluted p-values after applying the online p-value calculator were high as the first T-test (0.816559 for the test with the sub-datasets that contains ages less than 35 and 0.15937 for the test with the sub-datasets that contains ages more than 35) which also are not signifcant at ' p l.t 0.05 '. </p>

#### c.

In [ ]:
WhiteDf = newDataFrame[newDataFrame['Ethnicity_Modified'].eq('White')].sort_values(by=['Age'])
WhiteDf = WhiteDf.reset_index(drop=True)
OtherDf = newDataFrame[newDataFrame['Ethnicity_Modified'].eq('Other')].sort_values(by=['Age'])
OtherDf = OtherDf.reset_index(drop=True)

WhiteS=(len(WhiteDf)-1)*WhiteDf['Salary'].var()
OtherS = (len(OtherDf)-1)*OtherDf['Salary'].var()
DegOfFreeRace = (len(WhiteDf)+len(OtherDf))-2

PooledVarRace = (WhiteS+OtherS)/DegOfFreeRace
tDenominatorRace = math.sqrt(PooledVarRace/len(WhiteDf) + PooledVarRace/len(OtherDf))
T_ScoreRace = ((WhiteDf['Salary'].mean() - OtherDf['Salary'].mean())-0)/tDenominatorRace
#p_valueRace = 0.94984
#The result is not significant at p < .05

WhiteDfYoung = WhiteDf[WhiteDf['Age'].between(0,34)]
WhiteDfYoung = WhiteDfYoung.reset_index(drop=True)
WhiteDfOld = WhiteDf[WhiteDf['Age'].between(35,70)]
WhiteDfOld = WhiteDfOld.reset_index(drop=True)
OtherDfYoung = OtherDf[OtherDf['Age'].between(0,34)]
OtherDfYoung = OtherDfYoung.reset_index(drop=True)
OtherDfOld = OtherDf[OtherDf['Age'].between(35,70)]
OtherDfOld = OtherDfOld.reset_index(drop=True)

whiteYoungS=(len(WhiteDfYoung)-1)*WhiteDfYoung['Salary'].var()
otherYoungS = (len(OtherDfYoung)-1)*OtherDfYoung['Salary'].var()
DegOfFreeRaceYoung = (len(WhiteDfYoung)+len(OtherDfYoung))-2

PooledVarRaceYoung = (whiteYoungS+otherYoungS)/DegOfFreeRaceYoung
tDenominatorRaceY = math.sqrt(PooledVarRaceYoung/len(WhiteDfYoung) + PooledVarRaceYoung/len(OtherDfYoung))
T_ScoreRaceYoung = ((WhiteDfYoung['Salary'].mean() - OtherDfYoung['Salary'].mean())-0)/tDenominatorRaceY
#-------------------------------------
whiteOldS=(len(WhiteDfOld)-1)*WhiteDfOld['Salary'].var()
otherOldS = (len(OtherDfOld)-1)*OtherDfOld['Salary'].var()
DegOfFreeRaceOld = (len(WhiteDfOld)+len(OtherDfOld))-2

PooledVarRaceOld = (whiteOldS+otherOldS)/DegOfFreeRaceOld
tDenominatorRaceO = math.sqrt(PooledVarRaceOld/len(WhiteDfOld) + PooledVarRaceOld/len(OtherDfOld))
T_ScoreRaceOld = ((WhiteDfOld['Salary'].mean() - OtherDfOld['Salary'].mean())-0)/tDenominatorRaceO

<p>We repeated the same hypothesis testing we on the Gender but we used a different argument which is the Ethnicity, that makes our null hypothesis equals: µ WhiteSalary - µ OtherSalary = 0. The data was split into 2 sub-datasets based on the Ethnicity values (white, Other) and the T-test was made on these 2 independant datasets with unkown variance but assumed to be equal and degrees of freedom equals 172. the T-score for that test was equal '-0.66434'. Applying this value along with the degrees of freedon into the p-value calculator gave us the p-value '0.507579' which is also not sgnificant at 'p l.t 0.05'<br><br> After the first  general Ethnicity hypothesis testing now comes the time for the Ethnicity hypothesis testing with the Age factor added. The 2 Ethnicity based sub-datasets were splitted to 2 each, giving us 4 new sub-datasets which are "White l.t 35, white g.t 35, Other l.t 35, and Other g.t 35". Then the T-tests was made on these sub-datasets with degrees of freedom in the first test that equals '74' and degrees of freedom in the second test that equals '96'. The T-scores for the first test -the sub-dataset with ages under 35- was -2.15477 and by applying the values to the p-value calculator the p-value was equal 0.034497 which is significant at 'p l.t 0.05'. And the T-scores for the second test -the sub-dataset with ages over 35- was 0.811652 and by applying the values to the p-value calculator the p-value was equal 0.419029 which is not significant at 'p l.t 0.05'. <br><br> In conclusion, based on the results of the tests we reject the null hypothesis and say that there a discrimination in the company in both arguments the Age and the Ethnicity. The only accepted null hypothesis is the one fore the Ethnicity with ages below 35.</p>

<h2> Presentation video: <a href = 'https://livebueedu-my.sharepoint.com/:v:/g/personal/fares170916_bue_edu_eg/ERDcE7nB62FNhcHoirB1nTgBbT9U9zNW1le2bi1PPZzOhw?e=oUd9hw'>Click Here</a></h2>

<center><h1> Thank you and Happy Eid 😃 </h1></center>